In [8]:
import bs4, requests, re, os, time, json, random, urllib
import base64, hashlib
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.by import By

In [13]:
def wait_for_page_load(driver, timeout=10):
    start = time.time()
    while time.time() - start < timeout:
        if driver.execute_script("return document.readyState") == "complete":
            return True
        time.sleep(0.1)

In [17]:
import urllib.parse


url = 'https://mantarayadvocates.com/manta-ray-library/'

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver_service = Service(executable_path='/snap/bin/geckodriver')
browser = webdriver.Firefox(options=options, service=driver_service)

browser.get(url)

wait_for_page_load(browser)

manta_links = browser.find_elements(By.CLASS_NAME, "dp-dfg-image-link")

manta_links = [link.get_attribute('href') for link in manta_links]
manta_links = [urllib.parse.urlparse(link) for link in manta_links]

mantas = []

for link in manta_links:
    parts = link.path.strip('/').split('/')
    
    if len(parts) == 2 and parts[0] == 'manta-ray':
        name = parts[1].replace('-hawaii', '')

        print(f"Fetching images for '{name}' from {link.geturl()}")
        
        browser.get(link.geturl())
        wait_for_page_load(browser)
        
        # find img tags with class "skip-lazy" and download them
        images = browser.find_elements(By.CLASS_NAME, "skip-lazy")

        print(f"Found {len(images)} images")
        
        for image in images:
            src = image.get_attribute('src')
            if src:
                
                print(f"Image retrieved: {src}")
                
                mantas.append({
                    'name': name,
                    'image': src
                })

Fetching images for 'akari-ray' from https://mantarayadvocates.com/manta-ray/akari-ray-hawaii/
Found 0 images
Fetching images for 'alexander-ray' from https://mantarayadvocates.com/manta-ray/alexander-ray-hawaii/
Found 16 images
Image retrieved: 
Image retrieved: https://mantarayadvocates.com/wp-content/uploads/2018/09/Alexander-Ray-Images-2018-4.jpg
Image retrieved: https://mantarayadvocates.com/wp-content/uploads/2018/09/Alexander-Ray-Images-2018-3.jpg
Image retrieved: https://mantarayadvocates.com/wp-content/uploads/2018/09/Alexander-Ray-Images-2018-2.jpg
Image retrieved: https://mantarayadvocates.com/wp-content/uploads/2018/09/Alexander-Ray-Images-2018-1-scaled.jpg
Image retrieved: 
Image retrieved: https://mantarayadvocates.com/wp-content/uploads/2018/09/Alexander-Ray-Images-2018-4.jpg
Image retrieved: https://mantarayadvocates.com/wp-content/uploads/2018/09/Alexander-Ray-Images-2018-3.jpg
Image retrieved: https://mantarayadvocates.com/wp-content/uploads/2018/09/Alexander-Ray-Imag

In [18]:
import pandas as pd

# remove any image that starts with 'data:image'
mantas = [manta for manta in mantas if not manta['image'].startswith('data:image')]

df = pd.DataFrame(mantas)

df

,name,image
0,alexander-ray,https://mantarayadvocates.com/wp-content/uploa...
1,alexander-ray,https://mantarayadvocates.com/wp-content/uploa...
2,alexander-ray,https://mantarayadvocates.com/wp-content/uploa...
3,alexander-ray,https://mantarayadvocates.com/wp-content/uploa...
4,alexander-ray,https://mantarayadvocates.com/wp-content/uploa...
...,...,...
437,vegemite-ray,https://mantarayadvocates.com/wp-content/uploa...
438,vegemite-ray,https://mantarayadvocates.com/wp-content/uploa...
439,vegemite-ray,https://mantarayadvocates.com/wp-content/uploa...
440,vegemite-ray,https://mantarayadvocates.com/wp-content/uploa...


In [28]:
df.to_csv('mantas.csv', index=False)

In [30]:
with open('mantas.csv', 'r') as f:
    s3.put_object(
        Bucket='jam-personal-storage',
        Key='datasets/manta-rays-kona/mantas.csv',
        Body=f.read(),
        ContentType='text/csv'
    )

In [31]:
import tqdm
import requests
import boto3

session = boto3.Session(profile_name='juser')
s3 = session.client('s3')

loader = tqdm.tqdm(total=len(df))

for i, row in df.iterrows():
    image_name = row.image.split('/')[-1]
    key = f"datasets/manta-rays-kona/{row['name']}/{image_name}"
    
    # skip if image already exists
    try:
        s3.head_object(Bucket='jam-personal-storage', Key=key)
        loader.update(1)
        continue
    except:
        pass
    
    response = requests.get(row.image, stream=True, headers={'User-Agent': 'Mozilla/5.0'}, timeout=10)
    
    if response.status_code != 200:
        print(f"Failed to download image {row.image}")
        continue

    s3.put_object(
        Bucket='jam-personal-storage',
        Key=key,
        Body=response.content,
        ContentType='image/jpeg'
    )

    loader.update(1)

100%|██████████| 442/442 [11:34<00:00,  1.60s/it]